In [ ]:
%load_ext autoreload
from __future__ import print_function, division

In [ ]:
%autoreload
import copy, math, os, pickle, time, pandas as pd, numpy as np, scipy.stats as ss
import random

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import torch, torch.utils.data as utils, torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from torch.autograd import Variable
from torch.nn.parameter import Parameter

from sklearn.model_selection import StratifiedKFold

In [ ]:
'''
See github directions on getting data access from Physionet.org
'''
DATA_FILEPATH     = "./data/all_hourly_data.h5"
#RAW_DATA_FILEPATH = './all_hourly_data.h5'
GAP_TIME          = 6  # In hours
WINDOW_SIZE       = 24 # In hours
SEED              = 1
ID_COLS           = ['subject_id', 'hadm_id', 'icustay_id']
RESULTS_DIR     = "./results/"
GPU               = '2'

os.environ['CUDA_VISIBLE_DEVICES'] = GPU
np.random.seed(SEED)
torch.manual_seed(SEED)

def set_primary_seeds(seed):
    print("Setting primary seeds...")
    if not seed:
        seed = 1
        
    torch.manual_seed(SEED)
    np.random.seed(SEED)
#     np.random.RandomState(SEED)
    random.seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)  # for multiGPUs.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    
set_primary_seeds(SEED)

In [ ]:
'''
Some tools from: https://github.com/MLforHealth/MIMIC_Extract/tree/master/notebooks
'''
class DictDist():
    def __init__(self, dict_of_rvs): self.dict_of_rvs = dict_of_rvs
    def rvs(self, n):
        a = {k: v.rvs(n) for k, v in self.dict_of_rvs.items()}
        out = []
        for i in range(n): out.append({k: vs[i] for k, vs in a.items()})
        return out
    
class Choice():
    def __init__(self, options): self.options = options
    def rvs(self, n): return [self.options[i] for i in ss.randint(0, len(self.options)).rvs(n)]

In [ ]:
%%time
data_full_lvl2 = pd.read_hdf(DATA_FILEPATH, 'vitals_labs')
#data_full_raw  = pd.read_hdf(RAW_DATA_FILEPATH, 'vitals_labs') 
statics        = pd.read_hdf(DATA_FILEPATH, 'patients')

In [ ]:
'''
Some tools from: https://github.com/MLforHealth/MIMIC_Extract/tree/master/notebooks

“Simple Imputation” scheme outlined in Che et al.:

Zhengping Che, Sanjay Purushotham, Kyunghyun Cho, David Sontag, and Yan
Liu. 2018. Recurrent Neural Networks for Multivariate Time Series with Missing
Values. Scientific Reports 8, 1 (2018).

'''
def simple_imputer(df):
    idx = pd.IndexSlice
    df = df.copy()
    if len(df.columns.names) > 2: df.columns = df.columns.droplevel(('label', 'LEVEL1', 'LEVEL2'))
    
    df_out = df.loc[:, idx[:, ['mean', 'count']]]
    icustay_means = df_out.loc[:, idx[:, 'mean']].groupby(ID_COLS).mean()
    
    df_out.loc[:,idx[:,'mean']] = df_out.loc[:,idx[:,'mean']].groupby(ID_COLS).fillna(
        method='ffill'
    ).groupby(ID_COLS).fillna(icustay_means).fillna(0)
    
    df_out.loc[:, idx[:, 'count']] = (df.loc[:, idx[:, 'count']] > 0).astype(float)
    df_out.rename(columns={'count': 'mask'}, level='Aggregation Function', inplace=True)
    
    is_absent = (1 - df_out.loc[:, idx[:, 'mask']])
    hours_of_absence = is_absent.cumsum()
    time_since_measured = hours_of_absence - hours_of_absence[is_absent==0].fillna(method='ffill')
    time_since_measured.rename(columns={'mask': 'time_since_measured'}, level='Aggregation Function', inplace=True)

    df_out = pd.concat((df_out, time_since_measured), axis=1)
    df_out.loc[:, idx[:, 'time_since_measured']] = df_out.loc[:, idx[:, 'time_since_measured']].fillna(100)
    
    df_out.sort_index(axis=1, inplace=True)
    return df_out

In [ ]:
'''
Data preprocessing to define 3-day and 7-day length of stay outcome labels
'''
Ys = statics[statics.max_hours > WINDOW_SIZE + GAP_TIME][['mort_hosp', 'mort_icu', 'los_icu']]
Ys['los_3'] = Ys['los_icu'] > 3
Ys['los_7'] = Ys['los_icu'] > 7
# Ys.drop(columns=['los_icu'], inplace=True)
Ys.astype(float)

lvl2, raw = [df[
    (df.index.get_level_values('icustay_id').isin(set(Ys.index.get_level_values('icustay_id')))) &
    (df.index.get_level_values('hours_in') < WINDOW_SIZE)
] for df in (data_full_lvl2, data_full_lvl2)]

In [ ]:
# get subjects that were admited before and after 2180 [2100,2200]
Ys['admittime'] = statics[statics.max_hours > WINDOW_SIZE + GAP_TIME][['admittime']]
split_date = np.datetime64('2180-01-01')
Ys_int = Ys[Ys['admittime']<split_date]
Ys_ext = Ys[Ys['admittime']>=split_date]
subjects_int = Ys_int.index
subjects_ext = Ys_ext.index
del(Ys_int)
del(Ys_ext)
Ys.drop(labels='admittime',axis=1,inplace=True)
Ys.astype(float)

In [ ]:
lvl2_subj_idx,  Ys_subj_idx = [df.index.get_level_values('subject_id') for df in (lvl2, Ys)]
#lvl2_subj_idx, raw_subj_idx, Ys_subj_idx = [df.index.get_level_values('subject_id') for df in (lvl2, raw, Ys)]
lvl2_subjects = set(lvl2_subj_idx)
assert lvl2_subjects == set(Ys_subj_idx), "Subject ID pools differ!"
#assert lvl2_subjects == set(raw_subj_idx), "Subject ID pools differ!"

# shuffle the dataset
subjects, N = np.random.permutation(list(lvl2_subjects)), len(lvl2_subjects)

# standardize the whole dataset
idx = pd.IndexSlice
lvl2_means, lvl2_stds = lvl2.loc[:, idx[:,'mean']].mean(axis=0), lvl2.loc[:, idx[:,'mean']].std(axis=0)
lvl2.loc[:, idx[:,'mean']] = (lvl2.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds

# impute missing values
lvl2 = simple_imputer(lvl2)

idx = pd.IndexSlice
X_int = lvl2.loc[idx[subjects_int.get_level_values('subject_id')],:]
X_ext = lvl2.loc[idx[subjects_ext.get_level_values('subject_id')],:]
Y_int = Ys.loc[idx[subjects_int.get_level_values('subject_id')],:]
Y_ext = Ys.loc[idx[subjects_ext.get_level_values('subject_id')],:]

print("X internal shape: (%d,%d)"%(X_int.shape[0]/24,X_int.shape[1]))
print("X external shape: (%d,%d)"%(X_ext.shape[0]/24,X_ext.shape[1]))
print("Y internal shape: (%d,%d)"%(Y_int.shape[0],Y_int.shape[1]))
print("Y external shape: (%d,%d)"%(Y_ext.shape[0],Y_ext.shape[1]))

all_int_subjects = list(
    np.random.permutation(Y_int.index.get_level_values('subject_id').values)
)

print("X internal shape: (%d,%d)"%(X_int.shape[0]/24,X_int.shape[1]))
print("X external shape: (%d,%d)"%(X_ext.shape[0]/24,X_ext.shape[1]))
print("Y internal shape: (%d,%d)"%(Y_int.shape[0],Y_int.shape[1]))
print("Y external shape: (%d,%d)"%(Y_ext.shape[0],Y_ext.shape[1]))

In [ ]:
def run_basic(model, hyperparams_list, X_train, Ys_train, X_dev, Ys_dev, X_test, Ys_test, target):
    best_s, best_hyperparams, best_hyperparams_i = -np.Inf, None, None
    for i, hyperparams in enumerate(hyperparams_list):
        print("On sample %d / %d (hyperparams = %s)" % (i+1, len(hyperparams_list), repr((hyperparams))))
        M = model(**hyperparams)
        M = M.fit(X_train, Ys_train[target])
        y_score = M.predict_proba(X_dev)[:, 1]
        s = roc_auc_score(Ys_dev[target], y_score)
        if s > best_s:
            best_s, best_hyperparams, best_hyperparams_i  = s, hyperparams, i
            print("New Best AUC Score: %.2f @ hyperparams = %s" % (100*best_s, repr((best_hyperparams))))
    return run_only_final(model, best_hyperparams, best_hyperparams_i, X_train, Ys_train, X_dev, Ys_dev, X_test, Ys_test, target)

def run_only_final(model, best_hyperparams, best_hyperparams_i, X_train, Ys_train, X_dev, Ys_dev, X_test, Ys_test, target, pass_thru_model=False):
    if not pass_thru_model:
        best_M = model(**best_hyperparams)
    else:
        best_M = model
        
    print(X_train.shape)
    print(X_dev.shape)
    print(Ys_train.shape)
    print(Ys_dev.shape)
    best_M.fit(pd.concat((X_train, X_dev)), pd.concat((Ys_train, Ys_dev))[target])
    y_true  = Ys_test[target]
    y_score = best_M.predict_proba(X_test)[:, 1]
    y_pred  = best_M.predict(X_test)

    auc   = roc_auc_score(y_true, y_score)
    auprc = average_precision_score(y_true, y_score)
    acc   = accuracy_score(y_true, y_pred)
    F1    = f1_score(y_true, y_pred)
    
    return best_M, best_hyperparams, best_hyperparams_i, auc, auprc, acc, F1, y_true, y_score

In [ ]:
'''
Some helper data structures to store and save predictions
'''
class Logger():
    def __init__(self, optional_cols=None):
        #self.n_samples=n_samples
        self.columns=['task_name','fold','prediction_no','index','y_true','y_score','censoring']
        if (optional_cols is None):
            self.df=pd.DataFrame(columns=self.columns)
            self.has_optional_cols=False
        else:
            self.df=pd.DataFrame(columns=self.columns+optional_cols)
            self.has_optional_cols=True
            self.optional_cols=optional_cols
        self._rocs=[]
        self._prediction_no=0
        return
    
    def append_logger(self,indices, y_true, y_score, label, censoring=None, optional_dict=None,fold=0):
        y_true=np.array(y_true).astype(int)
        y_score=np.array(y_score).astype(float)
        

        if ((y_true.shape[0]!=y_score.shape[0])):
            raise ValueError("Shapes of input matrices must match")
        
            
        self._n=y_true.shape[0]

        if(censoring is None):
            cens = self._n*[math.nan]
            censoring=np.array(censoring)
        else:
            cens=censoring
        
        arr=np.array([self._n*[label],
                      self._n*[fold],
                      self._n*[self._prediction_no],
                      list(indices),
                      list(y_true),
                      list(y_score),
                      list(cens)
              ]).transpose()

        to_append=pd.DataFrame(arr, columns=self.columns)
        if(self.has_optional_cols):
            
            for column, value in optional_dict.items():
                to_append.loc[:,column]=value

        self.df=self.df.append(to_append)
        self._prediction_no=self._prediction_no+1
        
def preds_df_to_int(df):
    df_test = df
    type_dict = {}
    cast=['fold','prediction_no','y_true']
    for col in cast:
        type_dict[col] = 'int64'
    df_test = df_test.astype(dtype=type_dict)
    return df_test

## Logistic Regression (LR)

In [ ]:
N = 5

LR_dist = DictDist({
    'C': Choice(np.geomspace(1e-3, 1e3)),
    'penalty': Choice(['l1', 'l2']),
    'solver': Choice(['liblinear']),
    'max_iter': Choice([100])
})
np.random.seed(SEED)
LR_hyperparams_list = LR_dist.rvs(N)
for i in range(N):
    if LR_hyperparams_list[i]['solver'] == 'lbfgs': LR_hyperparams_list[i]['penalty'] = 'l2'

In [ ]:
LR_hyperparams_list

In [ ]:
RESULTS_DIR     = "/home/bryanbed/Projects/DevNotebooks/results/"
#with open(RESULTS_PATH, mode='rb') as f: results = pickle.load(f)
#RESULTS_PATH = './results/regression_baseline_res.pkl'
RERUN = True
results = dict()

In [ ]:
# demo LogisticRegression Classifier with LOS_3
# auc, auprc, acc, F1
early_stop_frac = 0.1
sss = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)
outcomes = ['los_3', 'los_7', 'mort_icu', 'mort_hosp']
model_name = 'LR'
model = LogisticRegression
hyperparams_list = LR_hyperparams_list
preds_int = Logger()
preds_ext = Logger()
for t in outcomes:
    print("Outcome:", t)
    Y_int[t] = Y_int[t].astype(int)
    Y_ext[t] = Y_ext[t].astype(int)
    fold=1
    best_fold_F1, best_fold_rmse, best_fold_auc, best_fold_auprc = -np.Inf, -np.Inf, -np.Inf, -np.Inf
    best_fold_model_name = "N/A"
    best_fold = -1
    best_fold_model = None
    for train_index, test_index in sss.split(idx[subjects_int.get_level_values('subject_id')], Y_int[t]):
        # Internal: 10-fold cross validation for training split
        best_F1, best_rmse, best_auc, best_auprc = -np.Inf, -np.Inf, -np.Inf, -np.Inf
        best_hyperparams = None
        best_preds = []
        best_model = None
        print("Evaluating %s for Outcome: %s, Fold: %d"%(model_name,t,fold))
        #train_data = subjects
        train_subj = subjects_int[train_index].get_level_values('subject_id')
        test_subj = subjects_int[test_index].get_level_values('subject_id')
        [(X_train, X_test), (Ys_train, Ys_test)] = [
            [df[df.index.get_level_values('subject_id').isin(s)] for s in (train_subj,  test_subj)] \
            for df in (X_int, Y_int)
        ]
        for df in X_train, X_test, Ys_train, Ys_test: assert not df.isnull().any().any()
            
        set_primary_seeds(SEED)
        all_train_subjects = list(
            np.random.permutation(Ys_train.index.get_level_values('subject_id').values)
        )
        N_early_stop        = int(len(all_train_subjects) * early_stop_frac)
        train_subjects      = all_train_subjects[:-N_early_stop]
        early_stop_subjects = all_train_subjects[-N_early_stop:]

        X_train_obs         = X_train[X_train.index.get_level_values('subject_id').isin(train_subjects)]
        Ys_train_obs        = Ys_train[Ys_train.index.get_level_values('subject_id').isin(train_subjects)]
        X_train_early_stop  = X_train[X_train.index.get_level_values('subject_id').isin(early_stop_subjects)]
        Ys_train_early_stop = Ys_train[Ys_train.index.get_level_values('subject_id').isin(early_stop_subjects)]
        
        print("Train subjects length: ", len(train_subjects))
        print("ES/valid subjects length: ", len(early_stop_subjects))   
        print("Test subjects length: ", len(test_subj))
              
        print("Running model %s on target %s" % (model_name, t))
        
        print(X_train_obs.shape)

        X_train_obs, X_train_early_stop, X_test = [
            df.pivot_table(index=['subject_id', 'hadm_id', 'icustay_id'], columns=['hours_in']) for df in (
                X_train_obs, X_train_early_stop, X_test
            )
        ]
        
        best_model, best_hyperparams_int, best_hyperparams_i, best_auc, auprc, acc, F1, y_true, y_score = run_basic(
            model, hyperparams_list, X_train_obs, Ys_train_obs, X_train_early_stop, Ys_train_early_stop, X_test, Ys_test, t
        )
        print("Final results for model %s on outcome %s" % (model_name, t))
        print("auc->%f, auprc->%f, acc->%f, F1->%f" % (best_auc, auprc, acc, F1))

        print("New Best AUC within fold %s: %.2f @ hyperparams = %s" % (str(fold), 100*best_auc, repr((best_hyperparams_int))))
#         best_true = y_true
#         best_preds = y_pred
              
        if best_auc > best_fold_auc:
            best_fold_auc, best_fold_hyperparams = best_auc, best_hyperparams_int
#             best_fold_model_name = model_name
            print("New Best AUC across all folds: %.2f @ hyperparams = %s" % (100*best_fold_auc, repr((best_fold_hyperparams))))
#             save our best model just in case we want it later
#             torch.save(best_model.module.state_dict(), 'results/best_overall_model.pt')
            best_fold = fold
            best_fold_model = best_model
        assert len(y_true) == len(y_score), "Labels (%d) and preds lengths (%d) dont match"%(len(y_true),len(y_score))
        print("Appending best predictions from Fold %d. Length: %d"%(fold, len(y_score)))
        subject_idx = list(range(0,len(y_true)))
        preds_int.append_logger(subject_idx, y_true, y_score, label=t, fold=fold)
        fold+=1
        print()
    # external
    set_primary_seeds(SEED)
    all_train_subjects = list(
        np.random.permutation(Y_int.index.get_level_values('subject_id').values)
    )
    N_early_stop        = int(len(all_train_subjects) * early_stop_frac)
    train_subjects      = all_train_subjects[:-N_early_stop]
    early_stop_subjects = all_train_subjects[-N_early_stop:]

    X_train_obs         = X_int[X_int.index.get_level_values('subject_id').isin(train_subjects)]
    Ys_train_obs        = Y_int[Y_int.index.get_level_values('subject_id').isin(train_subjects)]
    X_train_early_stop  = X_int[X_int.index.get_level_values('subject_id').isin(early_stop_subjects)]
    Ys_train_early_stop = Y_int[Y_int.index.get_level_values('subject_id').isin(early_stop_subjects)]
    X_test = X_ext
    Ys_test = Y_ext

    X_train_obs, X_train_early_stop, X_test = [
        df.pivot_table(index=['subject_id', 'hadm_id', 'icustay_id'], columns=['hours_in']) for df in (
            X_train_obs, X_train_early_stop, X_test
        )
    ]
    
    _, _, _, auc, auprc, acc, F1, y_true, y_score = run_only_final(
        best_fold_model, best_fold_hyperparams, 1, X_train_obs, Ys_train_obs, X_train_early_stop, Ys_train_early_stop, X_test, Ys_test, t, pass_thru_model=True
    )
    print("External Test Results: auc->%f, auprc->%f, acc->%f, F1->%f" % (auc, auprc, acc, F1))
    assert len(y_true) == len(y_score), "Labels (%d) and preds lengths (%d) dont match"%(len(y_true),len(y_score))
    print("Appending best overall predictions. Length: %d"%(len(y_score)))
    subject_idx = list(range(0,len(y_true)))
    preds_ext.append_logger(subject_idx, y_true, y_score, label=t, fold=fold)
    print()

preds_int.df = preds_df_to_int(preds_int.df)
preds_int.df.to_csv(RESULTS_DIR+"LR_internal_test_preds.csv")
preds_ext.df = preds_df_to_int(preds_ext.df)
preds_ext.df.to_csv(RESULTS_DIR+"LR_external_test_preds.csv")